# Collecting data on decentralized exchanges using BitQuery GraphQL API

**First we define how to run a query using API key from BitQuery**

In [6]:
import requests
import pandas as pd
import yaml


with open('api.yaml', 'r') as f:
    parsed_yaml = yaml.load(f, Loader=yaml.FullLoader)
    API_KEY = parsed_yaml['api']


def run_query(query):  # A simple function to use requests.post to make the API call.
    headers = {'X-API-KEY': API_KEY}
    request = requests.post('https://graphql.bitquery.io/', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed and return code is {}.      {}'.format(request.status_code, query))


In [7]:
query5 = """
{
   ethereum {
    dexTrades(options: {desc: ["date.year"]}) {
      count
      tradeAmount(in: USD)
      date {
        year
      }
    }
  }
}
"""
result5 = run_query(query5)  # Execute the query
print ('Result - {}'.format(result5))

Result - {'data': {'ethereum': {'dexTrades': None}}, 'errors': [{'message': 'Net::ReadTimeout with #<TCPSocket:(closed)>', 'locations': [{'line': 4, 'column': 5}], 'path': ['ethereum', 'dexTrades'], 'error_type': 'client'}]}


In [8]:
# Convert the result to a dataframe
df5 = pd.json_normalize(result5['data']['ethereum']['dexTrades'])
print(df5)

NotImplementedError: 

**The below query fetches the top decentralized exchanges based on their protocol and sorted on the number of trades**

In [ ]:
query1 = """
{
  ethereum {
    dexTrades(options: {limit: 100, desc: "count"}, time: {since: "2022-01-01"}) {
      count
      protocol
    }
  }
}

"""
result1 = run_query(query1)  # Execute the query
print ('Result - {}'.format(result1))

In [ ]:
import matplotlib

# Convert the result to a dataframe
df1 = pd.json_normalize(result1['data']['ethereum']['dexTrades'])
print(df1)

plot1 = df1.plot.pie(y='count', subplots=True, figsize=(11,6))

**This is a good place to start as it shows us the most popular decentralized exchanges we can now compare**

**The below query gives the daily trade volume of USDT and ETH on Uniswap**

In [ ]:
query2 = """
{
  ethereum(network: ethereum) {
    dexTrades(
      options: {asc: "date.month", limit: 100}
      protocol: {is: "Uniswap v2"}
    ) {
      count(time: {since: "2022-01-01"})
      tradeAmount(in: USD)
      protocol
      date {
        month
        year
      }
    }
  }
}
"""
result2 = run_query(query2)  # Execute the query
print ('Result - {}'.format(result2))

In [ ]:
# Convert the result to a dataframe
df2 = pd.json_normalize(result2['data']['ethereum']['dexTrades'])
print(df2)

In [ ]:
query3 = """
{
  ethereum {
    dexTrades(
      options: {desc: "tradeAmount" limit: 100}
    	exchangeName: {not: ""}
      date: {since: "2022-01-01" till: null}
    ) {
      protocol
      exchange {
        name
      }
      tradeAmount(in:USD)
    }
  }
}
"""
result3 = run_query(query3)  # Execute the query
print ('Result - {}'.format(result3))

In [ ]:
# Convert the result to a dataframe
df3 = pd.json_normalize(result3['data']['ethereum']['dexTrades'])
print(df3)

plot3 = df3.head(6).plot.pie(title='Trade Amount USD of Top DEX since 01-Jan-2022',
                             y='tradeAmount', subplots=True, figsize=(11,6),
                             labels=df3['exchange.name'])

In [ ]:
query4 = """
{
  ethereum(network: ethereum) {
    dexTrades(
      options: {asc: "date.month", limit: 100}
      protocol: {is: "Uniswap v3"}
    ) {
      count
      tradeAmount(in: USD)
      protocol
      date {
        month
        year
      }
    }
  }
}

"""
result4 = run_query(query4)  # Execute the query
print ('Result - {}'.format(result4))

In [ ]:
# Convert the result to a dataframe
df4 = pd.json_normalize(result4['data']['ethereum']['dexTrades'])
print(df4)